In [1]:
#全局变量
hub_token = open('/root/hub_token.txt').read().strip()
repo_id = 'lansinuote/nlp.3.reading_for_understanding'
push_to_hub = True

In [2]:
from transformers import AutoTokenizer

#加载分词工具
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer('What is your name?', 'My name is Sylvain.')

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [3]:
#可以直接用我处理好的数据集,这段代码可以跳过不看
#从官方教程里抄出来的函数,总之就是squad数据的处理函数,过程非常复杂,即使是官方的实现也是有问题的,我实在没本事写这个
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation='only_second',
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length',
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char
                    and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[
                        token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(
                    token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


#虚拟一批数据
datas = {
    'id': [
        '570d88cdfed7b91900d461f1', '57268432f1498d1400e8e25e',
        '570d675db3d812140066d84e'
    ],
    'title':
    ['United_States_Army', 'East_India_Company', 'Franco-Prussian_War'],
    'context': [
        "The United States Army is made up of three components: the active component, the Regular Army; and two reserve components, the Army National Guard and the Army Reserve. Both reserve components are primarily composed of part-time soldiers who train once a month, known as battle assemblies or unit training assemblies (UTAs), and conduct two to three weeks of annual training each year. Both the Regular Army and the Army Reserve are organized under Title 10 of the United States Code, while the National Guard is organized under Title 32. While the Army National Guard is organized, trained and equipped as a component of the U.S. Army, when it is not in federal service it is under the command of individual state and territorial governors; the District of Columbia National Guard, however, reports to the U.S. President, not the district's mayor, even when not federalized. Any or all of the National Guard can be federalized by presidential order and against the governor's wishes.",
        'The East India Company\'s arms, granted in 1698, were: "Argent a cross Gules; in the dexter chief quarter an escutcheon of the arms of France and England quarterly, the shield ornamentally and regally crowned Or." The crest was: "A lion rampant guardant Or holding between the forepaws a regal crown proper." The supporters were: "Two lions rampant guardant Or, each supporting a banner erect Argent, charged with a cross Gules." The motto was AUSPICIO REGIS ET SENATUS ANGLIÆ (Latin: By right of the King and the Senate of England).',
        "With the defeat of the First Army, Prince Frederick Charles ordered a massed artillery attack against Canrobert's position at St. Privat to prevent the Guards attack from failing too. At 19:00 the 3rd Division of Fransecky's II Corps of the Second Army advanced across Ravine while the XII Corps cleared out the nearby town of Roncourt and with the survivors of the 1st Guards Infantry Division launched a fresh attack against the ruins of St. Privat. At 20:00, the arrival of the Prussian 4th Infantry Division of the II Corps and with the Prussian right flank on Mance Ravine, the line stabilised. By then, the Prussians of the 1st Guards Infantry Division and the XII and II Corps captured St. Privat forcing the decimated French forces to withdraw. With the Prussians exhausted from the fighting, the French were now able to mount a counter-attack. General Bourbaki, however, refused to commit the reserves of the French Old Guard to the battle because, by that time, he considered the overall situation a 'defeat'. By 22:00, firing largely died down across the battlefield for the night. The next morning, the French Army of the Rhine, rather than resume the battle with an attack of its own against the battle-weary German armies, retreated to Metz where they were besieged and forced to surrender two months later."
    ],
    'question': [
        'What are UTAs?', 'how many lions were on the EIC arms',
        'The surving soldiers of the 1st Guards Infantry Division launched a new attack against what?'
    ],
    'answers': [{
        'text': ['unit training assemblies'],
        'answer_start': [292]
    }, {
        'text': ['Two'],
        'answer_start': [330]
    }, {
        'text': ['ruins of St. Privat'],
        'answer_start': [431]
    }]
}

#调用squad数据预处理函数
features = prepare_train_features(datas)

#先看看处理后的结果
for k, v in features.items():
    print(k, len(v), v)
    print()

#还原成文字查看,很显然,即使是huggingface的实现也是有问题的
for i in range(len(features['input_ids'])):
    input_ids = features['input_ids'][i]
    start_positions = features['start_positions'][i]
    end_positions = features['end_positions'][i]

    print('问题和文本')
    question_and_context = tokenizer.decode(input_ids)
    print(question_and_context)

    print('答案')
    answer = tokenizer.decode(input_ids[start_positions:end_positions])
    print(answer)

    print('原答案')
    original_answer = datas['answers'][i]['text'][0]
    print(original_answer)
    print()

input_ids 3 [[101, 2054, 2024, 28981, 2015, 1029, 102, 1996, 2142, 2163, 2390, 2003, 2081, 2039, 1997, 2093, 6177, 1024, 1996, 3161, 6922, 1010, 1996, 3180, 2390, 1025, 1998, 2048, 3914, 6177, 1010, 1996, 2390, 2120, 3457, 1998, 1996, 2390, 3914, 1012, 2119, 3914, 6177, 2024, 3952, 3605, 1997, 2112, 1011, 2051, 3548, 2040, 3345, 2320, 1037, 3204, 1010, 2124, 2004, 2645, 17720, 2030, 3131, 2731, 17720, 1006, 28981, 2015, 1007, 1010, 1998, 6204, 2048, 2000, 2093, 3134, 1997, 3296, 2731, 2169, 2095, 1012, 2119, 1996, 3180, 2390, 1998, 1996, 2390, 3914, 2024, 4114, 2104, 2516, 2184, 1997, 1996, 2142, 2163, 3642, 1010, 2096, 1996, 2120, 3457, 2003, 4114, 2104, 2516, 3590, 1012, 2096, 1996, 2390, 2120, 3457, 2003, 4114, 1010, 4738, 1998, 6055, 2004, 1037, 6922, 1997, 1996, 1057, 1012, 1055, 1012, 2390, 1010, 2043, 2009, 2003, 2025, 1999, 2976, 2326, 2009, 2003, 2104, 1996, 3094, 1997, 3265, 2110, 1998, 7894, 11141, 1025, 1996, 2212, 1997, 3996, 2120, 3457, 1010, 2174, 1010, 4311, 2000, 1996,

[CLS] what are utas? [SEP] the united states army is made up of three components : the active component, the regular army ; and two reserve components, the army national guard and the army reserve. both reserve components are primarily composed of part - time soldiers who train once a month, known as battle assemblies or unit training assemblies ( utas ), and conduct two to three weeks of annual training each year. both the regular army and the army reserve are organized under title 10 of the united states code, while the national guard is organized under title 32. while the army national guard is organized, trained and equipped as a component of the u. s. army, when it is not in federal service it is under the command of individual state and territorial governors ; the district of columbia national guard, however, reports to the u. s. president, not the district's mayor, even when not federalized. any or all of the national guard can be federalized by presidential order and against th

In [4]:
from datasets import load_dataset


def get_dataset():
    #加载数据集
    dataset = load_dataset('squad')

    #采样,数据量太大了跑不动
    dataset['train'] = dataset['train'].shuffle().select(range(10000))
    dataset['validation'] = dataset['validation'].shuffle().select(range(200))

    print(dataset, dataset['train'][0])

    #应用预处理函数
    dataset = dataset.map(
        function=prepare_train_features,
        batched=True,
        remove_columns=['id', 'title', 'context', 'question', 'answers'])

    return dataset


if push_to_hub:
    dataset = get_dataset()
    dataset.push_to_hub(repo_id=repo_id, token=hub_token)

#直接使用我处理好的数据集
dataset = load_dataset(path=repo_id)

print(dataset, dataset['train'][0])

Found cached dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 200
    })
}) {'id': '572eae2203f989190075692b', 'title': 'Neptune', 'context': 'An alternative concept is that they formed closer to the Sun, where the matter density was higher, and then subsequently migrated to their current orbits after the removal of the gaseous protoplanetary disc. This hypothesis of migration after formation is favoured, due to its ability to better explain the occupancy of the populations of small objects observed in the trans-Neptunian region. The current most widely accepted explanation of the details of this hypothesis is known as the Nice model, which explores the effect of a migrating Neptune and the other giant planets on the structure of the Kuiper belt.', 'question': "What is the most widely accepted explanati

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Generating validation split:   0%|          | 0/205 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10106 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--nlp.3.reading_for_understanding-461fa2d0775783fc/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 205
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10106
    })
}) {'input_ids': [101, 2054, 2003, 1996, 2087, 4235, 3970, 7526, 1997, 21167, 1005, 1055, 4195, 2170, 1029, 102, 2019, 4522, 4145, 2003, 2008, 2027, 2719, 3553, 2000, 1996, 3103, 1010, 2073, 1996, 3043, 4304, 2001, 3020, 1010, 1998, 2059, 3525, 13447, 2000, 2037, 2783, 20347, 2044, 1996, 8208, 1997, 1996, 3806, 14769, 15053, 11751, 18219, 5860, 1012, 2023, 10744, 1997, 9230, 2044, 4195, 2003, 16822, 1010, 2349, 2000, 2049, 3754, 2000, 2488, 4863, 1996, 1051, 9468, 6279, 11656, 1997, 1996, 7080, 1997, 2235, 5200, 5159, 1999, 1996, 9099, 1011, 11265, 13876, 19496, 2319, 2555, 1012, 1996, 2783, 2087, 4235, 3970, 7526, 1997, 1996, 4751, 1997, 2023, 10744, 2003, 2124, 2004, 1996, 3835, 2944, 1010, 202

In [5]:
import torch
from transformers.data.data_collator import default_data_collator

#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=default_data_collator,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

len(loader), data

(1263,
 {'input_ids': tensor([[ 101, 2054, 3663,  ...,    0,    0,    0],
          [ 101, 2029, 6328,  ...,    0,    0,    0],
          [ 101, 2040, 2106,  ...,    0,    0,    0],
          ...,
          [ 101, 6904, 2818,  ...,    0,    0,    0],
          [ 101, 2029, 2095,  ...,    0,    0,    0],
          [ 101, 2054, 2020,  ...,    0,    0,    0]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]]),
  'start_positions': tensor([54, 42, 42, 54, 64, 31, 14, 58]),
  'end_positions': tensor([55, 47, 44, 55, 65, 31, 14, 59])})

In [6]:
from transformers import AutoModelForQuestionAnswering, DistilBertModel, PreTrainedModel, PretrainedConfig

#加载模型
#model = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased')


#定义下游任务模型
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.pretrained = DistilBertModel.from_pretrained(
            'distilbert-base-uncased')

        self.fc = torch.nn.Sequential(torch.nn.Dropout(0.1),
                                      torch.nn.Linear(768, 2))

        #加载预训练模型的参数
        parameters = AutoModelForQuestionAnswering.from_pretrained(
            'distilbert-base-uncased')
        self.fc[1].load_state_dict(parameters.qa_outputs.state_dict())

    def forward(self, input_ids, attention_mask, start_positions,
                end_positions):
        #[b, lens] -> [b, lens, 768]
        logits = self.pretrained(input_ids=input_ids,
                                 attention_mask=attention_mask)
        logits = logits.last_hidden_state

        #[b, lens, 768] -> [b, lens, 2]
        logits = self.fc(logits)

        #[b, lens, 2] -> [b, lens, 1],[b, lens, 1]
        start_logits, end_logits = logits.split(1, dim=2)

        #[b, lens, 1] -> [b, lens]
        start_logits = start_logits.squeeze(2)
        end_logits = end_logits.squeeze(2)

        #起点和终点都不能超出句子的长度
        lens = start_logits.shape[1]
        start_positions = start_positions.clamp(0, lens)
        end_positions = end_positions.clamp(0, lens)

        criterion = torch.nn.CrossEntropyLoss(ignore_index=lens)

        start_loss = criterion(start_logits, start_positions)
        end_loss = criterion(end_logits, end_positions)
        loss = (start_loss + end_loss) / 2

        return {
            'loss': loss,
            'start_logits': start_logits,
            'end_logits': end_logits
        }


model = Model(PretrainedConfig())

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

out = model(**data)

out['loss'], out['start_logits'].shape, out['end_logits'].shape

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transfo

6636.4418


(tensor(6.0223, grad_fn=<DivBackward0>),
 torch.Size([8, 384]),
 torch.Size([8, 384]))

In [7]:
#测试
def test():
    model.eval()

    #数据加载器
    loader_val = torch.utils.data.DataLoader(
        dataset=dataset['validation'],
        batch_size=16,
        collate_fn=default_data_collator,
        shuffle=True,
        drop_last=True,
    )

    start_offset = 0
    end_offset = 0
    total = 0
    for i, data in enumerate(loader_val):
        #计算
        with torch.no_grad():
            out = model(**data)

        start_offset += (out['start_logits'].argmax(dim=1) -
                         data['start_positions']).abs().sum().item()

        end_offset += (out['end_logits'].argmax(dim=1) -
                       data['end_positions']).abs().sum().item()

        total += 16

        if i % 10 == 0:
            print(i)

        if i == 50:
            break

    print(start_offset / total, end_offset / total)

    start_logits = out['start_logits'].argmax(dim=1)
    end_logits = out['end_logits'].argmax(dim=1)
    for i in range(4):
        input_ids = data['input_ids'][i]

        pred_answer = input_ids[start_logits[i]:end_logits[i]]

        label_answer = input_ids[
            data['start_positions'][i]:data['end_positions'][i]]

        print('input_ids=', tokenizer.decode(input_ids))
        print('pred_answer=', tokenizer.decode(pred_answer))
        print('label_answer=', tokenizer.decode(label_answer))
        print()
        

test()

0
10
66.60416666666667 59.317708333333336
input_ids= [CLS] what kind of people attend the ipcc meetings? [SEP] the ipcc panel is composed of representatives appointed by governments and organizations. participation of delegates with appropriate expertise is encouraged. plenary sessions of the ipcc and ipcc working groups are held at the level of government representatives. non governmental and intergovernmental organizations may be allowed to attend as observers. sessions of the ipcc bureau, workshops, expert and lead authors meetings are by invitation only. attendance at the 2003 meeting included 350 government officials and climate change experts. after the opening ceremonies, closed plenary sessions were held. the meeting report states there were 322 persons in attendance at sessions with about seven - eighths of participants being from governmental organizations. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [8]:
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    model.to(device)
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
            
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            lr = optimizer.state_dict()['param_groups'][0]['lr']

            start_offset = (out['start_logits'].argmax(dim=1) -
                            data['start_positions']).abs().sum().item() / 8

            end_offset = (out['end_logits'].argmax(dim=1) -
                          data['end_positions']).abs().sum().item() / 8

            print(i, loss.item(), lr, start_offset, end_offset)

    model.to('cpu')
    
if push_to_hub:
    train()
    model.push_to_hub(repo_id=repo_id, use_auth_token=hub_token)

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 5.937665939331055 1.9984164687252575e-05 48.5 61.25
50 4.27445125579834 1.9192399049881235e-05 55.125 46.0
100 4.810187816619873 1.84006334125099e-05 64.375 63.0
150 3.7016162872314453 1.760886777513856e-05 30.625 28.75
200 3.559271812438965 1.6817102137767223e-05 36.5 36.5
250 1.9991931915283203 1.6025336500395887e-05 12.375 6.0
300 2.82735013961792 1.5233570863024545e-05 34.875 19.25
350 2.7468807697296143 1.4441805225653207e-05 14.375 19.5
400 2.095215320587158 1.365003958828187e-05 15.5 17.75
450 2.3270087242126465 1.2858273950910532e-05 29.125 16.375
500 1.429112195968628 1.2066508313539194e-05 0.125 3.625
550 2.092491388320923 1.1274742676167856e-05 13.75 17.625
600 1.715018630027771 1.0482977038796518e-05 9.25 3.125
650 2.470383405685425 9.69121140142518e-06 28.5 23.875
700 1.7490839958190918 8.899445764053842e-06 14.125 7.125
750 1.6417120695114136 8.107680126682502e-06 6.25 11.0
800 2.0058679580688477 7.315914489311164e-06 8.75 10.375
850 2.267788887023926 6.5241488519398265

pytorch_model.bin:   0%|          | 0.00/265M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
#直接使用我训练好的模型
model = Model.from_pretrained(repo_id)

test()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transfo

0
10
15.546875 18.494791666666668
input_ids= [CLS] what is the universal band that digital recievers will receive free to air channels on? [SEP] provided is a universal ku band lnb ( 9. 75 / 10. 600 ghz ) which is fitted at the end of the dish and pointed at the correct satellite constellation ; most digital receivers will receive the free to air channels. some broadcasts are free - to - air and unencrypted, some are encrypted but do not require a monthly subscription ( known as free - to - view ), some are encrypted and require a monthly subscription, and some are pay - per - view services. to view the encrypted content a videoguard uk equipped receiver ( all of which are dedicated to the sky service, and cannot be used to decrypt other services ) needs to be used. unofficial cams are now available to view the service, although use of them breaks the user's contract with sky and invalidates the user's rights to use the card. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 